In [2]:
from utils.dataset import  CustomDataset
import os
import warnings
warnings.filterwarnings("ignore")
import glob
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import sys
import pandas as pd
from tqdm import tqdm
import threading
import random
# import rasterio
import os
import numpy as np
import sys
from sklearn.utils import shuffle as shuffle_lists
import numpy as np
from sklearn.model_selection import train_test_split
import joblib

import torch

#default
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#trasnform
from torchvision import transforms

#dataset
from utils.dataset import CustomDataset
from torch.utils.data import DataLoader, random_split
import torchvision.models as models
from torchsummary import summary

from utils import models 
#metric
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import torchmetrics.functional as tf

#numeric
import numpy as np
import pandas as pd 

#visualization
import matplotlib.pyplot as plt

#system 
from tqdm import tqdm
import os 
import wandb
import datetime


In [3]:
now = datetime.datetime.now()
weekday = now.strftime("%A")  # %A는 현재 요일을 나타내는 포맷 지정자입니다.
formatted_now = now.strftime("%Y%m%d%H%M")
checkpoint_datetime = formatted_now
print(f"#"*30, "SAVE TIME", "#"*30)
print(f"#"*30, checkpoint_datetime, "#"*30)

############################## SAVE TIME ##############################
############################## 202403140852 ##############################


In [9]:
train_csv = '../../dataset/train_meta.csv'

train_transform = transforms.Compose([
    # transforms.Resize((512,512), antialias= True),
    # transforms.RandomApply([
    #     transforms.RandomRotation(degrees = 30),
    #     transforms.RandomVerticalFlip(p= 0.5),
    #     transforms.RandomVerticalFlip(p=0.5),
    #     transforms.RandomCrop(size =224, scale= (0.8, 1.2), ratio = (0.75, 1.2), antialias = True),
    #     transforms.ColorJitter(brightness= 0.2, contrast = 0.2)
    # ]),
    transforms.ToTensor()
])

tr_batch = 2
vl_batch = 2

dataset = CustomDataset(
    csv_path= train_csv,
    transform= None, #None으로 세팅
    MAX_PIXEL_VALUE= 65535,
    band = (7,6,2) #기존 세팅 
)

# 훈련 및 검증 세트 분할
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size 
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])
train_loader = DataLoader(dataset = train_dataset, batch_size = tr_batch, shuffle= True)
valid_loader = DataLoader(dataset = valid_dataset, batch_size = vl_batch, shuffle= False)



In [8]:
for images, masks in train_loader:
    print(images.shape)
    idx = 0
    plt.figure(figsize=(6,4))
    plt.subplot(121)
    plt.imshow(images[idx])
    plt.title('image')
    plt.subplot(122)
    plt.imshow(masks[idx], cmap= 'gray')
    plt.title('mask')
    plt.show()
    break

for images, masks in valid_loader:
    print(images.shape)
    idx = 0
    plt.figure(figsize=(6,4))
    plt.subplot(121)
    plt.imshow(images[idx])
    plt.title('image')
    plt.subplot(122)
    plt.imshow(masks[idx], cmap= 'gray')
    plt.title('mask')
    plt.show()
    break

RasterioIOError: '../../dataset/train_img/train_img_10872.tif' not recognized as a supported file format.

In [34]:
# cost func & optimizer 정의

# random seed 고정
random.seed(42)
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.cuda.manual_seed_all(42)
else:
    device = torch.device('cuda')
    torch.cuda.manual_seed_all(42)


model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)

lr = 0.0001
optimizer = torch.optim.AdamW(model.parameters(), lr = lr)

loss = nn.BCELoss()

print(f'device : {device}')
summary(model, input_size=(3, 224, 224))



Using cache found in /Users/eiden/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


device : cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             864
       BatchNorm2d-2         [-1, 32, 224, 224]              64
              ReLU-3         [-1, 32, 224, 224]               0
            Conv2d-4         [-1, 32, 224, 224]           9,216
       BatchNorm2d-5         [-1, 32, 224, 224]              64
              ReLU-6         [-1, 32, 224, 224]               0
         MaxPool2d-7         [-1, 32, 112, 112]               0
            Conv2d-8         [-1, 64, 112, 112]          18,432
       BatchNorm2d-9         [-1, 64, 112, 112]             128
             ReLU-10         [-1, 64, 112, 112]               0
           Conv2d-11         [-1, 64, 112, 112]          36,864
      BatchNorm2d-12         [-1, 64, 112, 112]             128
             ReLU-13         [-1, 64, 112, 112]               0
        MaxPool2d-14     

In [32]:
metrics = {
    'tr_bce' : [],
    'vl_bce' : []
}
epoch, epochs = 0, 200
early_stopping_epochs, early_stopping_cnt= 10, 0
best_loss = float('inf')

In [43]:
for epoch in tqdm(range(epoch, epochs)):
    train_losses, valid_losses = 0., 0.
    
    model.train()
    for _, (images, masks) in tqdm(enumerate(train_loader)):
        images, masks = images.to(device), masks.to(device)
        preds = model(images)
        
        train_loss = loss(preds, masks).to(device)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        train_losses += train_loss.item()
        
        
    with torch.no_grad():
        model.eval()
        for _, (images, masks) in tqdm(enumerate(train_loader)):
            images, masks = images.to(device), masks.to(device)
            preds = model(images)
            
            valid_loss = loss(preds, masks).to(device)
            valid_losses += valid_loss.item()
            
    metrics['tr_bce'].append(train_losses / len(train_loader))
    metrics['vl_bce'].append(valid_losses / len(valid_loader))
    
    # Early Stopping
    if valid_losses < best_loss:
        best_loss = valid_losses
        early_stopping_cnt = 0
    else:
        early_stopping_cnt += 1
        if early_stopping_cnt >= early_stopping_epochs:
            print(f"Early Stops!!! : {epoch}/{epochs}")
    
    # Validation 성능이 이전보다 우수한 경우 모델 저장
    if valid_losses > np.array(metrics['vl_bce'])[:-1].max():
        SAVE_DIR = '../result'
        torch.save({
            "model" : f"{epoch}",
            "epoch" : epoch,
            "epochs" : epochs,
            "model_state_dict" : model.state_dict(),
            "optimizer_state_dict" : optimizer.state_dict(),
            "learning_rate" : lr,
            "loss" : loss,
            "metric" : metrics,
            "description" : f"segmentation model training status : {epoch}/{epochs}"
        },
        f'../model/{checkpoint_datetime}.pt')
        
        
        
        
        
            
        

0it [00:00, ?it/s]/200 [00:00<?, ?it/s]
  0%|          | 0/200 [00:00<?, ?it/s]


AssertionError: Torch not compiled with CUDA enabled